In [1]:
!pip install numpy==1.24.4 pandas==1.5.3 openpyxl==3.0.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
pywavelets 1.9.0 requires numpy<3,

In [1]:
import openpyxl

# Create a new workbook for order details data
wb_order_details = openpyxl.Workbook()
ws_order_details = wb_order_details.active
ws_order_details.title = "受注明細" # シート名を「受注明細」に変更

# Add headers to the order details sheet
ws_order_details['A1'] = '商品名'
ws_order_details['B1'] = '単価'
ws_order_details['C1'] = '数量' # 数量のヘッダーを追加

# Add some sample product data with quantities
# 商品名, 単価, 数量
ws_order_details['A2'] = '商品A'
ws_order_details['B2'] = 10000
ws_order_details['C2'] = 2 # 数量を追加

ws_order_details['A3'] = '商品B'
ws_order_details['B3'] = 15000
ws_order_details['C3'] = 1 # 数量を追加

ws_order_details['A4'] = '商品C'
ws_order_details['B4'] = 5000
ws_order_details['C4'] = 3 # 数量を追加

# Save the order details workbook
wb_order_details.save("受注明細.xlsx")
print("受注明細.xlsx が作成されました。")

受注明細.xlsx が作成されました。


In [3]:
import openpyxl
from datetime import date # datetimeモジュールからdateをインポート

# --- Load Order Details Data ---
# 受注明細.xlsx が存在するか確認し、読み込む
try:
    wb_order_details = openpyxl.load_workbook("受注明細.xlsx", data_only=True)
    ws_order_details = wb_order_details["受注明細"]  # 実データのシート名に変更
    ws_order_details = wb_order_details.active
    order_items_from_file = [] # 受注明細から読み込んだ商品リスト
    # A列(商品名), B列(単価), C列(数量)からデータを読み込む (ヘッダー行はスキップ)
    for row in ws_order_details.iter_rows(min_row=2, values_only=True):
        if row[0] is not None: # 商品名が空でない場合のみ処理
            product_name = row[0]
            unit_price = row[1]
            quantity = row[2] # 数量をC列から読み込む
            order_items_from_file.append({
                'name': product_name,
                'unit_price': unit_price,
                'quantity': quantity
            })
except FileNotFoundError:
    print("エラー: '受注明細.xlsx' が見つかりません。")
    print("先に受注明細作成スクリプトを実行してファイルを作成してください。")
    exit() # ファイルが見つからない場合は処理を終了

# Create a new workbook and select the active worksheet for the invoice
wb = openpyxl.Workbook()
ws = wb.active

# --- Header Section ---
# Title: "請求書" (Invoice)
ws['B2'] = '請求書'
ws.merge_cells('B2:E2')
ws['B2'].font = openpyxl.styles.Font(size=14, bold=True)

# Company Information
ws['B4'] = '株式会社ABC'
ws['B5'] = '〒101-0022 東京都千代田区神田練塀町300'
ws['B6'] = 'TEL: 03-1234-5678 FAX: 03-1234-5678'
ws['B7'] = '担当者名: 鈴木一郎 様'

# Invoice Details (No. and Date)
ws['F4'] = 'No.'
ws['G4'] = '0001'
ws['F5'] = '日付'

# Get the current date and format it as YYYY/MM/DD
today = date.today()
formatted_date = today.strftime("%Y/%m/%d")
ws['G5'] = formatted_date # Format the date as YYYY/MM/DD and assign to cell

# --- Table Header ---
ws['B10'] = '商品名'
ws['C10'] = '数量'
ws['D10'] = '単価'
ws['E10'] = '金額'

# Apply formatting to table headers
header_cells = ['B10', 'C10', 'D10', 'E10']
for cell in header_cells:
    ws[cell].border = openpyxl.styles.Border(
        left=openpyxl.styles.Side(style='thin'),
        right=openpyxl.styles.Side(style='thin'),
        top=openpyxl.styles.Side(style='thin'),
        bottom=openpyxl.styles.Side(style='thin')
    )

# --- Table Data ---
# 受注明細ファイルから読み込んだデータを使用して請求書の明細を生成
start_row = 11 # 明細の開始行
current_row = start_row
for item in order_items_from_file:
    product_name = item['name']
    quantity = item['quantity']
    unit_price = item['unit_price']

    ws[f'B{current_row}'] = product_name
    ws[f'C{current_row}'] = quantity
    ws[f'D{current_row}'] = unit_price
    # 数式で金額を計算 (C列の数量 * D列の単価)
    ws[f'E{current_row}'] = f'={ws.cell(row=current_row, column=3).coordinate}*{ws.cell(row=current_row, column=4).coordinate}'

    current_row += 1

# --- Summary Section ---
# 合計、消費税、税込合計の行を動的に設定
# 明細の最終行の次の行から合計を開始
total_row = current_row # 明細の最終行が合計行になる
ws[f'D{total_row}'] = '合計'
# 明細の金額列の範囲を動的に設定
ws[f'E{total_row}'] = f'=SUM(E{start_row}:E{current_row-1})'
ws[f'D{total_row}'].border = openpyxl.styles.Border(bottom=openpyxl.styles.Side(style='thin'))
ws[f'E{total_row}'].border = openpyxl.styles.Border(bottom=openpyxl.styles.Side(style='thin'))

# 消費税と税込合計
ws[f'D{total_row + 2}'] = '合計'
ws[f'E{total_row + 2}'] = f'=SUM(E{start_row}:E{current_row-1})' # 上の合計と同じ数式
ws[f'D{total_row + 3}'] = '消費税'
ws[f'E{total_row + 3}'] = f'={ws.cell(row=total_row + 2, column=5).coordinate}*0.1' # 10% tax
ws[f'D{total_row + 4}'] = '税込合計'
ws[f'E{total_row + 4}'] = f'={ws.cell(row=total_row + 2, column=5).coordinate}+{ws.cell(row=total_row + 3, column=5).coordinate}'

# --- Final Adjustments ---
# Set column widths for better readability
ws.column_dimensions['B'].width = 15
ws.column_dimensions['C'].width = 10
ws.column_dimensions['D'].width = 10
ws.column_dimensions['E'].width = 15
ws.column_dimensions['G'].width = 15

# Save the workbook
wb.save("請求書.xlsx")
print("請求書.xlsx が作成されました。")

請求書.xlsx が作成されました。
